# Data cleaning
Most of the demographic data comes from Eurostat, so they share similar format and basic cleaning can be generalized.  
A first selection of variables can already be done when downloading the data.  
Then, some specific operations may be necessary according to the original dataset and the final usage.   
The algorithms will use the cleaned datasets, but the original ones are kept in case different usages are defined.

In [1]:
import pandas as pd

In [2]:
data_folder = './'#'../data/'
age_path = data_folder + 'age/age_nuts3.csv'
employment_rate_path = data_folder + 'employment/employment_rate_nuts2.csv'
employment_type_path = data_folder + 'employment/employment_thousand_person_nuts3.csv'
income_path = data_folder + 'income/households_income_nuts2.csv'
motorization_path = data_folder + 'motorization/motorization_rate.csv'

## Basic functions

In [3]:
# filter by year
def filter_year(df, year):
    df = df[df['TIME_PERIOD']==year]
    df = df.drop(columns='TIME_PERIOD')
    return df.reset_index(drop=True)

In [4]:
# filter by max precision
def filter_precision(df):
    df = df[df['geo'].apply(len)<=5]
    max_precision = df['geo'].apply(len).max()
    df = df[df['geo'].apply(len)==max_precision]
    return df.reset_index(drop=True), max_precision-2

In [5]:
# compare data availability for different times
# NUTS 2: 242 regions
# NUTS 3: 1166 regions
def compare_years(df):
    return df.groupby('TIME_PERIOD').count()['OBS_VALUE']

In [6]:
# rename geo and OBS_VALUE columns
def rename_cols(df, obs_value_name, resolution):
    df = df.rename(columns={'geo': f'nuts{resolution}', 'OBS_VALUE': obs_value_name})
    return df

## Age dataset
| Unit | Resolution | Categories | 
|---|---|---|
|Count | NUTS-3 | 3 |

Age distribution, we keep 3 categories: under 20, between 20-64, and over 64

In [28]:
# load dataset
df_age = pd.read_csv(age_path)
# drop useless columns
df_age.drop(columns=['STRUCTURE', 'STRUCTURE_ID', 'freq', 'sex', 'unit', 'OBS_FLAG'], inplace=True)
df_age.head()

,age,geo,TIME_PERIOD,OBS_VALUE
0,UNK,AL,2021,0
1,UNK,AL,2022,0
2,UNK,AL0,2021,0
3,UNK,AL01,2021,0
4,UNK,AL011,2021,0


In [29]:
# filter by time
year = 2022

df_age = filter_year(df_age, year)
df_age, resolution = filter_precision(df_age)

In [30]:
# unknown age is always 0, so it can be dropped
print(df_age.loc[df_age['age']=='UNK', 'OBS_VALUE'].unique())
df_age = df_age[df_age['age']!='UNK']

[0]


In [31]:
df_age = rename_cols(df_age, 'count', resolution)

In [32]:
df_age['age'].unique()

array(['Y10-14', 'Y15-19', 'Y20-24', 'Y25-29', 'Y30-34', 'Y35-39',
       'Y40-44', 'Y45-49', 'Y5-9', 'Y50-54', 'Y55-59', 'Y60-64', 'Y65-69',
       'Y70-74', 'Y75-79', 'Y80-84', 'Y85-89', 'Y_GE85', 'Y_GE90',
       'Y_LT5'], dtype=object)

In [33]:
# regroup age ranges
under_20 = ['Y_LT5', 'Y5-9', 'Y10-14', 'Y15-19']
between_20_64 = ['Y20-24', 'Y25-29', 'Y30-34', 'Y35-39', 'Y40-44', 'Y45-49', 'Y50-54', 'Y55-59', 'Y60-64']
over_64 = ['Y65-69', 'Y70-74', 'Y75-79', 'Y80-84', 'Y85-89', 'Y_GE90']

In [34]:
df_under_20 = df_age[df_age['age'].isin(under_20)].groupby('nuts3').sum().reset_index()
df_under_20.rename(columns={'count':'age_under_20'}, inplace=True)

df_between_20_64 = df_age[df_age['age'].isin(between_20_64)].groupby('nuts3').sum().reset_index()
df_between_20_64.rename(columns={'count':'age_between_20_64'}, inplace=True)

df_over_64 = df_age[df_age['age'].isin(over_64)].groupby('nuts3').sum().reset_index()
df_over_64.rename(columns={'count':'age_over_64'}, inplace=True)

In [35]:
df_age_export = pd.merge(df_under_20, df_between_20_64, on='nuts3')
df_age_export = pd.merge(df_age_export, df_over_64, on='nuts3')
df_age_export.head()

,nuts3,age_under_20,age_between_20_64,age_over_64
0,AL011,29897,62959,16729
1,AL012,66842,178378,46105
2,AL013,19868,40965,11935
3,AL014,28043,69084,20156
4,AL015,44214,119305,29490


In [147]:
# export clean table to csv
df_age_export.to_csv(data_folder+f'age/age_nuts3_{year}_clean.csv', index=False)

## Employment rate dataset
| Unit | Resolution | Categories | 
|---|---|---|
|Percentage | NUTS-2 | 2 |

Categories: between 20-64, and over 64

In [33]:
# load dataset
df_employment = pd.read_csv(employment_rate_path)
# drop useless columns
df_employment.drop(columns=['STRUCTURE', 'STRUCTURE_ID', 'freq', 'sex', 'unit', 'OBS_FLAG'], inplace=True)
df_employment.head()

,age,geo,TIME_PERIOD,OBS_VALUE
0,Y20-64,AT,2021,75.6
1,Y20-64,AT,2022,77.3
2,Y20-64,AT1,2021,73.2
3,Y20-64,AT1,2022,74.6
4,Y20-64,AT11,2021,74.9


In [34]:
# filter by time
df_employment = filter_year(df_employment, year)

# filter by max precision (NUTS 2)
df_employment, resolution = filter_precision(df_employment)

In [35]:
df_employment = rename_cols(df_employment, 'employment_rate', resolution)

In [42]:
df_between_20_64 = df_employment[df_employment['age']=='Y20-64'].groupby('nuts2').sum().reset_index()
df_between_20_64.rename(columns={'employment_rate':'employment_rate_between_20_64'}, inplace=True)

df_over_64 = df_employment[df_employment['age']=='Y_GE65'].groupby('nuts2').sum().reset_index()
df_over_64.rename(columns={'employment_rate':'employment_rate_over_64'}, inplace=True)

In [43]:
df_employment_export = pd.merge(df_between_20_64, df_over_64, on='nuts2')
df_employment_export.head()

,nuts2,employment_rate_between_20_64,employment_rate_over_64
0,AT11,76.5,0.0
1,AT12,78.5,4.2
2,AT13,71.2,5.7
3,AT21,75.8,4.3
4,AT22,77.6,5.1


In [44]:
# export clean table to csv
df_employment_export.to_csv(data_folder+f'employment_rate/employment_rate_nuts2_{year}_clean.csv', index=False)

## Employment type dataset
| Unit | Resolution | Categories | 
|---|---|---|
|Percentage | NUTS-3 | 10 |

In [148]:
# load dataset
df_employment_type = pd.read_csv(employment_type_path)
# drop useless columns
df_employment_type.drop(columns=['STRUCTURE', 'STRUCTURE_ID', 'freq', 'unit', 'OBS_FLAG'], inplace=True)
df_employment_type.head()

,wstatus,nace_r2,geo,TIME_PERIOD,OBS_VALUE
0,EMP,A,AT,2020,152.3
1,EMP,A,AT1,2020,45.9
2,EMP,A,AT11,2020,7.4
3,EMP,A,AT111,2020,0.8
4,EMP,A,AT112,2020,4.4


In [149]:
# filters
df_employment_type, resolution = filter_precision(df_employment_type)
year=2020
df_employment_type = filter_year(df_employment_type, year)
df_employment_type = rename_cols(df_employment_type, 'employment_thousand_persons', resolution)
# keep only 'employed person' as it contains 'employee'
df_employment_type = df_employment_type[df_employment_type['wstatus']=='EMP']
df_employment_type.drop(columns='wstatus', inplace=True)

In [150]:
# pivot
df_employment_type_export = df_employment_type.pivot(index='nuts3', columns='nace_r2', values='employment_thousand_persons' )
df_employment_type_export.reset_index(inplace=True)
df_employment_type_export = df_employment_type_export.rename_axis(None, axis=1)
df_employment_type_export.drop(columns=['C', 'G-J', 'K-N', 'O-U'], inplace=True)

In [151]:
df_employment_type_export = df_employment_type_export.rename(columns={
    'A': 'Agriculture',
    'B-E': 'Industry',
    #'C': 'Manufacturing',
    'F': 'Construction',
    'G-I': 'Wholesale, retail trade, transport, accomodation and food service',
    'J': 'Information and communication',
    'K': 'Finance, insurance',
    'L': 'Real estate',
    'M_N': 'Professional, scientific and technical, administrative',
    'O-Q': 'Public administration, defence, education, health, social',
    'R-U': 'Arts, entertainment, other service',
    })

In [152]:
df_employment_type_export.head()

,nuts3,Agriculture,Industry,Construction,"Wholesale, retail trade, transport, accomodation and food service",Information and communication,"Finance, insurance",Real estate,"Professional, scientific and technical, administrative","Public administration, defence, education, health, social","Arts, entertainment, other service",TOTAL
0,AT111,0.8,2.5,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.8
1,AT112,4.4,8.9,5.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.1
2,AT113,2.2,5.8,3.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.5
3,AT121,8.3,27.7,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.3
4,AT122,4.3,22.3,8.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.1


Some data is missing but can be inferred from the informed data  
For now, a mean imputation is performed, it can later be replaced by a regression model, or crossing other datasets

In [178]:
for c in df_employment_type_export.columns:
    print(len(df_employment_type_export[c].dropna()), end = ' ')

1198 1190 1187 1194 692 693 694 694 697 701 697 1198 

In [159]:
def df_to_percent(df, index):
    '''
    Replace values by percentages per row
    Args:
        df (DataFrame): the dataframe to transform
        index (str): the name of the column to consider as index
    Returns:
        the same dataframe with values in percentages 
    '''
    df = df.set_index(index)
    df = df.div(df.sum(axis=1), axis=0)
    return df.reset_index()

In [162]:
# drop TOTAL to compute percentages
df_employment_type_percent = df_employment_type_export.copy().dropna().drop(columns='TOTAL')
df_employment_type_percent = df_to_percent(df_employment_type_percent, 'nuts3')

In [177]:
# plot some distributions to observe that it is coherent to keep the mean
import plotly.express as px
fig = px.histogram(df_employment_type_percent, x="Construction")
fig.show()

In [191]:
# Get mean percentage for each category, filled data only
total = 0
means = {}
columns = df_employment_type_percent.drop(columns='nuts3').columns
for c in columns:
    print(c)
    m = df_employment_type_percent[c].mean()
    print(m)
    means[c] = m
    total+=m
print('Total')
print(total) # close enough to 1

Agriculture
0.09009986708858476
Industry
0.17484548960927426
Construction
0.07035949998458924
Wholesale, retail trade, transport, accomodation and food service
0.23597104240588468
Information and communication
0.017415446429920073
Finance, insurance
0.016115915837179928
Real estate
0.00810170877034172
Professional, scientific and technical, administrative
0.09554233373474198
Public administration, defence, education, health, social
0.24199782038556797
Arts, entertainment, other service
0.049550875753915216
Total
0.9999999999999998


In [192]:
# replace missing values
# mean percentage of the colum times the total number of employed person for that region 
for c in columns:
    df_employment_type_export[c] = df_employment_type_export[c].fillna(means[c])*df_employment_type_export['TOTAL']

In [195]:
# back to percentages
df_employment_type_percent = df_employment_type_export.copy().dropna().drop(columns='TOTAL')
df_employment_type_percent = df_to_percent(df_employment_type_percent, 'nuts3')

In [197]:
# final result
df_employment_type_percent.head()

,nuts3,Agriculture,Industry,Construction,"Wholesale, retail trade, transport, accomodation and food service",Information and communication,"Finance, insurance",Real estate,"Professional, scientific and technical, administrative","Public administration, defence, education, health, social","Arts, entertainment, other service"
0,AT111,0.131911,0.412222,0.346266,0.038909,0.002872,0.002657,0.001336,0.015754,0.039903,0.008170
1,AT112,0.221498,0.448031,0.297009,0.011879,0.000877,0.000811,0.000408,0.004810,0.012182,0.002494
2,AT113,0.175094,0.461611,0.310394,0.018780,0.001386,0.001283,0.000645,0.007604,0.019260,0.003944
3,AT121,0.181760,0.606596,0.197089,0.005167,0.000381,0.000353,0.000177,0.002092,0.005299,0.001085
4,AT122,0.120230,0.623520,0.237665,0.006598,0.000487,0.000451,0.000227,0.002671,0.006766,0.001385


In [199]:
# export clean table to csv
df_employment_type_percent.to_csv(data_folder+f'employment/employment_type_nuts{resolution}_{year}_clean.csv', index=False)

## Motorization rate dataset
| Unit | Resolution | Categories | 
|---|---|---|
|Probability | NUTS-2 | 6 |

Counts converted to number per inhabitants, i.e. probability for an inhabitant to own a particular type of vehicle.  
Inhabitants are over 20 years old.  
Note that the values are not percentages, but independant probabilities.  
For example a person can have a car and a moto, so the sum of all vehicle types per inhabitants is not 1.

In [81]:
# load dataset
df_motor = pd.read_csv(motorization_path)
# drop useless columns
df_motor.drop(columns=['STRUCTURE', 'STRUCTURE_ID', 'freq', 'unit', 'OBS_FLAG'], inplace=True)
df_motor.head()

,vehicle,geo,TIME_PERIOD,OBS_VALUE
0,BUS_TOT,AT11,2020,312
1,BUS_TOT,AT11,2021,298
2,BUS_TOT,AT12,2020,1060
3,BUS_TOT,AT12,2021,1062
4,BUS_TOT,AT13,2020,3999


In [82]:
df_motor, resolution = filter_precision(df_motor)

In [83]:
compare_years(df_motor)

TIME_PERIOD
2020    2476
2021    2127
Name: OBS_VALUE, dtype: int64

In [84]:
year=2020
df_motor = filter_year(df_motor, year)
df_motor = rename_cols(df_motor, 'count', resolution)

In [85]:
df_motor[df_motor.duplicated(subset=['vehicle', 'nuts2'], keep=False)].head()

,vehicle,nuts2,count
258,CAR,AT11,201548
259,CAR,AT12,1113849
260,CAR,AT13,718819
261,CAR,AT21,367676
262,CAR,AT22,775945


In [86]:
# cars are sometimes informed a second time with a vey small value
# the largest is kept
cars = df_motor[df_motor['vehicle']=='CAR'].groupby('nuts2').max().reset_index()
df_motor = df_motor[df_motor['vehicle']!='CAR']
df_motor = pd.concat([df_motor, cars])

In [94]:
# pivot
df_motor_pivot = df_motor.pivot(index='nuts2', columns='vehicle', values='count' )
df_motor_pivot.reset_index(inplace=True)
df_motor_pivot = df_motor_pivot.rename_axis(None, axis=1)

In [95]:
df_motor_pivot.head()

,nuts2,BUS_TOT,CAR,LOR,MOTO,SPE,TOT_X_TM,TRC,TRL_STRL,UTL
0,AT11,312.0,201548.0,21289.0,21086.0,34373.0,258669.0,1147.0,39249.0,56809.0
1,AT12,1060.0,1113849.0,111912.0,122197.0,164955.0,1395822.0,4046.0,210647.0,280913.0
2,AT13,3999.0,718819.0,73553.0,77348.0,9082.0,806368.0,915.0,36825.0,83550.0
3,AT21,389.0,367676.0,33709.0,45360.0,41721.0,444617.0,1122.0,60890.0,76552.0
4,AT22,1098.0,775945.0,75119.0,90892.0,104733.0,959772.0,2877.0,130035.0,182729.0


Get the number of inhabitants per nuts2 to compute the vehicle/inhabitant

In [96]:
df_inhabitants = df_age_export.copy()
df_inhabitants['nuts2'] = df_inhabitants['nuts3'].apply(lambda s: s[:-1])
df_inhabitants['over_20'] = df_inhabitants['age_between_20_64'] + df_inhabitants['age_over_64']
df_inhabitants = df_inhabitants[['nuts2', 'over_20']]
df_inhabitants = df_inhabitants.groupby('nuts2').sum().reset_index()
df_inhabitants.head()

,nuts2,over_20
0,AL01,595106
1,AL02,912101
2,AL03,647100
3,AT11,244814
4,AT12,1367530


In [97]:
df_motor_pivot = df_motor_pivot.merge(df_inhabitants, on="nuts2")
df_motor_pivot = df_motor_pivot.set_index('nuts2')
df_motor_export = df_motor_pivot.div(df_motor_pivot['over_20'], axis=0)
df_motor_export.reset_index(inplace=True)

Similarly as employment types, the missing values are imputed by the means  
Few missing values here

In [98]:
for c in df_motor_export.columns:
    print(len(df_motor_export[c].dropna()), end = ' ')

253 253 253 240 242 232 245 240 236 232 253 

In [110]:
df_motor_export_dropna = df_motor_export.copy().dropna()

In [112]:
# Get mean proba for each category, filled data only
means = {}
columns = df_motor_export.drop(columns='nuts2').columns
for c in columns:
    print(c)
    m = df_motor_export_dropna[c].mean()
    print(m)
    means[c] = m

BUS_TOT
0.003367424688423838
CAR
0.646616602172732
LOR
0.08771689963792716
MOTO
0.06667389928758831
SPE
0.01042398935399071
TOT_X_TM
0.7538501136393772
TRC
0.0057251977863044074
TRL_STRL
0.07617594103670804
UTL
0.10386608677822223
over_20
1.0


In [113]:
# replace missing values
for c in columns:
    df_motor_export[c] = df_motor_export[c].fillna(means[c])

In [114]:
# regroup lorries and road tractors 
# regroupment has to be done after replacing missing values, otherwise it can group NaN with a valid number
df_motor_export['vehicle_truck'] = df_motor_export['LOR'] + df_motor_export['TRC']
# drop total and trailers
df_motor_export.drop(columns=['TOT_X_TM', 'TRL_STRL', 'LOR', 'TRC', 'over_20'], inplace=True)
# rename columns
df_motor_export = df_motor_export.rename(columns={
    'CAR': 'vehicle_car',
    'MOTO': 'vehicle_moto',
    'SPE': 'vehicle_special',
    'BUS_TOT': 'vehicle_bus',
    'UTL': 'vehicle_utilities',
})

In [115]:
# final dataframe
df_motor_export.head()

,nuts2,vehicle_bus,vehicle_car,vehicle_moto,vehicle_special,vehicle_utilities,vehicle_truck
0,AT11,0.001274,0.823270,0.086131,0.140405,0.232050,0.091645
1,AT12,0.000775,0.814497,0.089356,0.120623,0.205416,0.084794
2,AT13,0.002563,0.460772,0.049581,0.005822,0.053557,0.047735
3,AT21,0.000840,0.793959,0.097950,0.090092,0.165306,0.075214
4,AT22,0.001069,0.755740,0.088525,0.102006,0.177971,0.075965


In [117]:
# export clean table to csv
df_motor_export.to_csv(data_folder+f'motorization/motorization_proba_nuts{resolution}_{year}_clean.csv', index=False)

## Income dataset
| Unit | Resolution |
|---|---|
|€ per inhabitant | NUTS-2 | 

household income in € per inhabitant for a year

In [209]:
# load dataset
df_income = pd.read_csv(income_path)
# drop useless columns
df_income.drop(columns=['STRUCTURE', 'STRUCTURE_ID', 'freq', 'unit', 'direct', 'na_item', 'OBS_FLAG'], inplace=True)
df_income.head()

,geo,TIME_PERIOD,OBS_VALUE
0,AT,2020,26900
1,AT1,2020,26600
2,AT11,2020,26400
3,AT12,2020,28100
4,AT13,2020,25400


In [210]:
df_income, resolution = filter_precision(df_income)
year=2020
df_income = filter_year(df_income, year)
df_income = rename_cols(df_income, 'household_income', resolution)

In [224]:
# some values are duplicated, or different for the same region
df_income = df_income.groupby('nuts2').mean().reset_index()

In [227]:
df_income.head()

,nuts2,household_income
0,AT11,25400.0
1,AT12,26350.0
2,AT13,23700.0
3,AT21,23950.0
4,AT22,24750.0


In [226]:
# export clean table to csv
df_income.to_csv(data_folder+f'income/households_income_nuts{resolution}_{year}_clean.csv', index=False)

## Test loading

In [ ]:
from loader import EurostatLoader

l = EurostatLoader('2021', ['FRJ13', 'FRJ12'])
l.get_all_data()